In [1]:
from fastai.vision.widgets import *

import numpy as np
import pandas as pd

import os

!pip install semtorch
from fastai.basics import *
from fastai.vision import models
from fastai.vision.all import *
from fastai.metrics import *
from fastai.data.all import *
from fastai.callback import *

# SemTorch
from semtorch import get_segmentation_learner

from pathlib import Path
import random

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [2]:
#My file path is being desing to look like this one. 
path = Path("../dataset2/")
#image is a normal microscope image. It's in png format. 
path_images = path/"Images"
#labels are the masks balck and white. In tif format. 
path_labels = path/"Labels"

test_name = "test"
manual_name="manual"

path_manual_img = path_images/manual_name
path_manual_lbl = path_labels/manual_name

In [3]:
path2 = path_images/"RT4"
path2

Path('../dataset2/Images/RT4')

In [4]:
number_of_the_seed = 100

random.seed(number_of_the_seed)
set_seed(number_of_the_seed)

In [5]:
fnames_manual = get_image_files(path_manual_img)
lbl_names_manual = get_image_files(path_manual_lbl)

In [6]:
get_y_fn = lambda x: path_labels/manual_name/f'Mask_{str(x.stem[-3:])}.tif'

In [7]:
def ParentSplitter(x):
    return Path(x).parent.name==test_name

In [8]:
from albumentations import (
    Compose,
    OneOf,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    Flip,
    Rotate,
    Transpose,
    CLAHE,
    ShiftScaleRotate
)

In [9]:
class SegmentationAlbumentationsTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug): 
        self.aug = aug
    def encodes(self, x):
        img,mask = x
        aug = self.aug(image=np.array(img), mask=np.array(mask))
        return PILImage.create(aug["image"]), PILMask.create(aug["mask"])
    
transformPipeline=Compose([
                        Flip(p=0.5),
                        Transpose(p=0.5),
                        Rotate(p=0.40,limit=10)
                    ],p=1)

transformPipeline=SegmentationAlbumentationsTransform(transformPipeline)

In [10]:
class TargetMaskConvertTransform(ItemTransform):
    def __init__(self): 
        pass
    def encodes(self, x):
        img,mask = x
        
        #Convert to array
        mask = np.array(mask)
        
        # Change 255 for 1
        mask[mask==255]=1
        
        # Back to PILMask
        mask = PILMask.create(mask)
        return img, mask

In [11]:
def hrnet_splitter(model):
    return [params(model.backbone), params(model.head)]

In [12]:
codes = np.loadtxt(path/'codes.txt',dtype=str)
name2id = {v:k for k,v in enumerate(codes)}
print(name2id)
void_code = name2id['NoSpheroid']

{'NoSpheroid': 0, 'Spheroid': 1}


In [13]:
def pixel_accuracy(inp, targ, axis):
    "Compute accuracy with `targ` when `pred` is bs * n_classes"
    pred,targ = flatten_check(inp.argmax(dim=axis), targ)
    
    #the mask is the targ pixels different from NoSpheroid
    mask = targ != void_code
    #[0]
    #when the predicted pixels and targets pixels named spheroid correlates, we calculate the mean. 
    return (pred[mask] == targ[mask]).float().mean()

In [14]:
manual_name="manual"
camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items=get_image_files,
                   get_y=get_y_fn,
                   splitter=RandomSplitter(valid_pct=0.1, seed=60),
                   item_tfms=[Resize((160,160)),TargetMaskConvertTransform(), transformPipeline],
                   batch_tfms=Normalize.from_stats(*imagenet_stats)
                  )

# Time to load the models 

Two options. If you are using UNet input the parameters of your models and run the cell below. 

In [ ]:
dls = camvid.dataloaders(path_manual_img, bs=32)
learn = unet_learner(dls, resnet34, metrics=[accuracy, Dice(), JaccardCoeff()], self_attention=True, act_cls=Mish, opt_func=opt, wd=1e-2)

If SemTorch library has been used, customize this piece of code. Please, bear in mind:
* Architecture 
* Backbone 
* WD 
* Cost function or loss function 

In [15]:
dls = camvid.dataloaders(path_images,bs=32)
learn = get_segmentation_learner(dls=dls, number_classes=2, segmentation_type="Semantic Segmentation",
                                 architecture_name="hrnet", backbone_name="hrnet_w48", 
                                 metrics=[partial(pixel_accuracy, axis=1), Dice(), JaccardCoeff()], wd=1e-3,
                                 splitter=hrnet_splitter, loss_func= DiceLoss(axis=1, smooth=1, reduction='mean', square_in_union=False)).to_fp16()
learn.freeze()

/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


In [16]:
learn.load("HRNet_Aug_wd3_model_2_hrnet_w48_bs_32_Dice_loss")
learn.validate()

/opt/conda/lib/python3.7/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 39, in fetch
    data = next(self.dataset_iter)
  File "/opt/conda/lib/python3.7/site-packages/fastai/data/load.py", line 118, in create_batches
    yield from map(self.do_batch, self.chunkify(res))
  File "/opt/conda/lib/python3.7/site-packages/fastcore/basics.py", line 217, in chunked
    res = list(itertools.islice(it, chunk_sz))
  File "/opt/conda/lib/python3.7/site-packages/fastai/data/load.py", line 133, in do_item
    try: return self.after_item(self.create_item(s))
  File "/opt/conda/lib/python3.7/site-packages/fastai/data/load.py", line 140, in create_item
    if self.indexed: return self.dataset[s or 0]
  File "/opt/conda/lib/python3.

Run this if you had built learners. It creates a datablock with the testing images. 

In [17]:
dblock = DataBlock(blocks=(ImageBlock),get_items = get_image_files, item_tfms=Resize((160,160)), batch_tfms=Normalize.from_stats(*imagenet_stats))
learn.dls =  dblock.dataloaders(path2, bs=7, num_workers=0)

NameError: name 'learn' is not defined

Finally to make prediction in bulk 

In [ ]:
import os 
import ntpath
for file in path.ls():
    #os.mkdir('Predicted_masks_HRNet')
    filename = os.path2.basename(file)
    if "jpg" in os.path2.basename(file):
        trial = ntpath.basename(file)
        #learn.inf
        preds =learn.inf.predict(file)
        pred_1 = preds[0][1]
        pred_arx = pred_1.numpy()
        rescaled = (755.0 / pred_arx.max() * (pred_arx - pred_arx.min())).astype(np.uint8)
        im = Image.fromarray(rescaled) 
        for i in filename:
            if i.isupper() == True:
                identificator = filename.split(i)[0]
                break
        name = identificator + "_Mask_" + trial[-6:-4] + ".tif"
        im.save(f"Predicted_masks_HRNet/{name}")